In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import cv2
import shutil

In [6]:

train_dir = r"C:/Users/kants/machineeee/wee_detection/train"
test_dir = r"C:/Users/kants/machineeee/wee_detection/test"

class_names = ["Weed", "Crop"] 
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)


In [7]:

if not os.path.exists(os.path.join(train_dir, "Weed")) or not os.path.exists(os.path.join(train_dir, "Crop")):
    for file in os.listdir(train_dir):
        if file.endswith(".jpg") or file.endswith(".png"):
            if "weed" in file.lower():
                shutil.move(os.path.join(train_dir, file), os.path.join(train_dir, "Weed", file))
            else:
                shutil.move(os.path.join(train_dir, file), os.path.join(train_dir, "Crop", file))


In [8]:

batch_size = 32  
num_epochs = 15  
image_size = (128, 128)  
num_classes = len(class_names)


In [9]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))


In [10]:

for layer in base_model.layers:
    layer.trainable = False


In [11]:

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
class_outputs = Dense(num_classes, activation='softmax')(x)


In [12]:

model = Model(inputs=base_model.input, outputs=class_outputs)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [13]:

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [14]:

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 3321 images belonging to 2 classes.


In [15]:

val_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_dataset = val_datagen.flow_from_directory(
    test_dir,  
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 245 images belonging to 2 classes.


In [16]:

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [17]:

history = model.fit(
    train_dataset,  
    epochs=num_epochs,
    validation_data=val_dataset,
    callbacks=[early_stop, checkpoint, reduce_lr]
)

c:\Users\kants\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.4984 - loss: 0.9533
Epoch 1: val_accuracy improved from -inf to 0.24490, saving model to best_model.h5


104/104 ━━━━━━━━━━━━━━━━━━━━ 62s 557ms/step - accuracy: 0.4984 - loss: 0.9525 - val_accuracy: 0.2449 - val_loss: 0.9412 - learning_rate: 0.0010
Epoch 2/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.5127 - loss: 0.7684
Epoch 2: val_accuracy improved from 0.24490 to 0.30612, saving model to best_model.h5


104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.5125 - loss: 0.7684 - val_accuracy: 0.3061 - val_loss: 0.8603 - learning_rate: 0.0010
Epoch 3/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.4964 - loss: 0.7412
Epoch 3: val_accuracy did not improve from 0.30612
104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 273ms/step - accuracy: 0.4962 - loss: 0.7412 - val_accuracy: 0.1469 - val_loss: 1.0098 - learning_rate: 0.0010
Epoch 4/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.5014 - loss: 0.7195
Epoch 4: val_accuracy improved from 0.30612 to 0.39592, saving model to best_model.h5


104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.5014 - loss: 0.7195 - val_accuracy: 0.3959 - val_loss: 0.7292 - learning_rate: 0.0010
Epoch 5/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.4930 - loss: 0.7136
Epoch 5: val_accuracy did not improve from 0.39592
104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 281ms/step - accuracy: 0.4930 - loss: 0.7136 - val_accuracy: 0.1469 - val_loss: 0.9125 - learning_rate: 0.0010
Epoch 6/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.4860 - loss: 0.7095
Epoch 6: val_accuracy did not improve from 0.39592
104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.4861 - loss: 0.7095 - val_accuracy: 0.1796 - val_loss: 0.7836 - learning_rate: 0.0010
Epoch 7/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.5101 - loss: 0.6989
Epoch 7: val_accuracy did not improve from 0.39592
104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 279ms/step - accuracy: 0.5101 - loss: 0.6989 - val_accuracy: 0.1510 - val_loss: 0.8181 - learning_rate: 0.0010
Epoch 8

104/104 ━━━━━━━━━━━━━━━━━━━━ 28s 273ms/step - accuracy: 0.5157 - loss: 0.6946 - val_accuracy: 0.4776 - val_loss: 0.6966 - learning_rate: 2.0000e-04
Epoch 10/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.5038 - loss: 0.6989
Epoch 10: val_accuracy did not improve from 0.47755
104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 282ms/step - accuracy: 0.5038 - loss: 0.6989 - val_accuracy: 0.1510 - val_loss: 0.7829 - learning_rate: 2.0000e-04
Epoch 11/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.5101 - loss: 0.6956
Epoch 11: val_accuracy did not improve from 0.47755
104/104 ━━━━━━━━━━━━━━━━━━━━ 28s 272ms/step - accuracy: 0.5102 - loss: 0.6956 - val_accuracy: 0.4612 - val_loss: 0.6945 - learning_rate: 2.0000e-04
Epoch 12/15
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.4931 - loss: 0.6975
Epoch 12: val_accuracy did not improve from 0.47755
104/104 ━━━━━━━━━━━━━━━━━━━━ 29s 276ms/step - accuracy: 0.4931 - loss: 0.6975 - val_accuracy: 0.2939 - val_loss: 0.7141 - learning_ra

In [ ]:

model.save('weed_detection_model.h5')

In [ ]:

model = tf.keras.models.load_model('weed_detection_model.h5')

In [ ]:

def detect_weed_center_and_distance(img_path, reference_object_width_cm, reference_object_width_pixels):
    """
    Detect the center of the weed and calculate its distance from the top-left corner in centimeters.
    
    Args:
        img_path (str): Path to the input image.
        reference_object_width_cm (float): Real-world width of a known object in the image (in cm).
        reference_object_width_pixels (float): Width of the known object in the image (in pixels).
    
    Returns:
        center (tuple): Coordinates of the weed's center (x, y) in pixels.
        distance_cm (float): Distance of the weed's center from the top-left corner in centimeters.
    """
    
    img = cv2.imread(img_path)
    img_resized = cv2.resize(img, image_size)
    img_normalized = img_resized / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)

    
    predictions = model.predict(img_expanded)
    predicted_class = np.argmax(predictions[0])

    if predicted_class == 0:  
        gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:

            largest_contour = max(contours, key=cv2.contourArea)

            
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                center_x = int(M["m10"] / M["m00"])
                center_y = int(M["m01"] / M["m00"])
                center = (center_x, center_y)

                pixels_per_cm = reference_object_width_pixels / reference_object_width_cm
                distance_cm_x = center_x / pixels_per_cm
                distance_cm_y = center_y / pixels_per_cm
                distance_cm = np.sqrt(distance_cm_x**2 + distance_cm_y**2)

                return center, distance_cm

    return None, None  


In [ ]:

def real_time_weed_detection(reference_object_width_cm, reference_object_width_pixels, confidence_threshold=0.7):
    """
    Run real-time weed detection and calculate the center and distance of the weed.
    
    Args:
        reference_object_width_cm (float): Real-world width of a known object in the image (in cm).
        reference_object_width_pixels (float): Width of the known object in the image (in pixels).
        confidence_threshold (float): Confidence threshold for weed detection.
    """
    cap = cv2.VideoCapture(0)  

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        
        frame_resized = cv2.resize(frame, image_size)
        frame_normalized = frame_resized / 255.0
        frame_expanded = np.expand_dims(frame_normalized, axis=0)

        predictions = model.predict(frame_expanded)
        predicted_class = np.argmax(predictions[0])
        confidence = np.max(predictions[0])

        if predicted_class == 0 and confidence >= confidence_threshold:  
            center, distance_cm = detect_weed_center_and_distance(frame_resized, reference_object_width_cm, reference_object_width_pixels)

            if center is not None:
                
                cv2.circle(frame_resized, center, 5, (0, 0, 255), -1)  
                cv2.putText(frame_resized, f"Weed Detected ({confidence:.2f})", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.putText(frame_resized, f"Center: {center}, Distance: {distance_cm:.2f} cm", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame_resized, f"No Weed Detected ({confidence:.2f})", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Real-Time Weed Detection', frame_resized)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:

reference_object_width_cm = 10.0 
reference_object_width_pixels = 100 

test_image_path = r"C:\Users\kants\machineeee\wee_detection\test\Weed\ridderzuring_3129_jpg.rf.d376edabc61dc552b7f6671377a0ec98.jpg"
center, distance_cm = detect_weed_center_and_distance(test_image_path, reference_object_width_cm, reference_object_width_pixels)

if center is not None:
    print(f"Weed detected! Center: {center}, Distance from top-left corner: {distance_cm:.2f} cm")
else:
    print("No weed detected.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 793ms/step
No weed detected.


In [ ]:

real_time_weed_detection(reference_object_width_cm, reference_object_width_pixels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'imread'
> Overload resolution failed:
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object
>  - Expected 'filename' to be a str or path-like object


: 